# Mass-merge two activities

This is a simple script that will take all activities of type "A" and relabel them as activities of type "B".

 1. make a list of all activities and their IDs
 2. build a dictionary of all the conversions you'd like to do
 3. get a list of all time entries and covert the ones in the list
 4. \[optional\] archive the (now unused) entries
 5. \[optional\] complete the same steps for archived activity ids

_If you have the repo but not the package, you need to be in the root directory_

In [ ]:
cd ..

In [ ]:
import os, json, time
import datetime
from timeular import timeular

Get the api key and secret on the [timeular account page](https://profile.timeular.com/#/app/account). There is no concept of an API app, just a per-user api token

In [ ]:
with open(".env") as f:
    os.environ.update(json.load(f))

api_key = os.environ.get("TIMEULAR_API_KEY")
api_secret = os.environ.get("TIMEULAR_API_SECRET")

In [ ]:
t = timeular.TimeularSession(api_key, api_secret, no_edit_mode=False)
# verbose=True will show all requests and more information for non-200 responses.
# can be toggled using t._VERBOSE = True

# 1. get all activities and their IDs

In [ ]:
spaces_by_id = {s['id']: s for s in t.list_spaces()}

In [ ]:
activities_by_id = {a['id']: a for a in t.list_activities()}

In [ ]:
activities_sorted_by_space = sorted(activities_by_id.values(), key=lambda e: e["spaceId"])

In [ ]:
cur_space = None
print("## ACTIVITIES ##\n")
for activity in activities_sorted_by_space:
    space_name = spaces_by_id[activity['spaceId']]['name']
    if space_name != cur_space:
        cur_space = space_name
        print(f"\n{space_name}")
    print(f"\t{activity['name']:.<45}{activity['id']}")

# 2. Create a dictionary with all the merges

In [ ]:
original_id_to_new_id_map = {
    1112690: 1149216,
    1039873: 1149273,
    997480:  1149199,
    997479:  1149198,
    828160:  1149212,
    809893:  1149208,
    794234:  1095713,
    794230:  1149211,
    794226:  1149200
}

# 3. Get a list of all the time entries

In [ ]:
entries = t.find_entries_in_range(datetime.datetime(2000,1,1), datetime.datetime.now())

In [ ]:
unknown_ids = set()
merge_count = 0
for entry in entries:
    a_id = int(entry['activityId'])
    if False and a_id in original_id_to_new_id_map:
        merge_count += 1
        t.edit_entry(entry['id'], activity_id=original_id_to_new_id_map[a_id])
        time.sleep(.2)
    if a_id not in activities_by_id:
        unknown_ids.add(a_id)
print(f"merged {merge_count} item. \n encountered {len(unknown_ids)} archived activities.")

## 4.\[optional\] archive the (now unused) activities

In [ ]:
for a in original_id_to_new_id_map.keys():
    t.archive_activity(a)

In [ ]:
# quick table display hack
from IPython.display import HTML, display
def show_table(data):
    display(HTML(f'''<table><tr><th>{"</th><th>".join(map(str,data[0]))}</th></tr>
    <tr>{"</tr><tr>".join([f"<td>{'</td><td>'.join(map(str,row))}</td>" for row in data[1:]])}</tr></table>'''))

## 5. \[optional\] discover any archived activities still in use and clear them out

Unfortunately there's no way to get archived activity information using the public API (reversing the "non-public" api is pretty guessable but maybe unstable?).

Just make a list of entries using archived activities (sorted by activity and date), and decide if there's any worth merging

In [ ]:
# i've found an iteractive approach helps, so here's a line to reload entries
entries = t.find_entries_in_range(datetime.datetime(2000,1,1), datetime.datetime.now())

In [ ]:
table = [['activity ID', "start time (UTC)", 'notes or comments']]
for entry in sorted(entries, key=lambda x: (x['activityId'], x['duration']['startedAt'])): 
    if entry['activityId'] not in activities_by_id:
        table.append([entry['activityId'],entry['duration']['startedAt'], entry['note']['text'] or ''])
show_table(table)

In [ ]:
achived_to_unarchived_map = {
    830198: 998703,
}
for entry in entries:
    a_id = int(entry['activityId'])
    if a_id in achived_to_unarchived_map:
        print(entry['note']['text'], entry['duration']['startedAt'])
        t.edit_entry(entry['id'], activity_id=achived_to_unarchived_map[a_id])
        time.sleep(.2)